<a href="https://colab.research.google.com/github/Angie-ch/waste-classification-/blob/test/waste_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import tensorflow as tf
#import tensorflow.contrib.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
from PIL import Image
from pathlib import Path
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!git clone https://github.com/Angie-ch/waste-classification-

Cloning into 'waste-classification-'...
remote: Enumerating objects: 783, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 783 (delta 0), reused 3 (delta 0), pack-reused 773 (from 1)
Receiving objects: 100% (783/783), 296.39 MiB | 24.00 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [31]:
import os
import shutil
import random
# Define the path to the dataset
dataset_dir = '/content/waste-classification-/waste dataset'

# Define the path to where the split datasets will be saved
train_dir = 'train_data'
test_dir = 'test_data'

# List of class names
class_names = ['glass', 'plastic', 'paper', 'metal']

# Create directories for train and test sets (if they do not exist)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subdirectories for each class in train and test directories (if they do not exist)
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Split the dataset for each class
for class_name in class_names:
    # Get the path to the subfolder inside each class
    class_folder = os.path.join(dataset_dir, class_name)

    # List subdirectories (e.g., "paper waste", "glass waste")
    subfolders = [f for f in os.listdir(class_folder) if os.path.isdir(os.path.join(class_folder, f))]

    for subfolder in subfolders:
        subfolder_path = os.path.join(class_folder, subfolder)

        # Get the list of image files in the current subfolder
        image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]

        # Shuffle the list of image files to ensure randomness
        random.shuffle(image_files)

        # Calculate the number of images for training (70%)
        total_images = len(image_files)
        train_size = int(0.7 * total_images)

        # Split the files into training and testing sets
        train_images = image_files[:train_size]
        test_images = image_files[train_size:]

        # Move the training images to the train directory
        for image in train_images:
            src = os.path.join(subfolder_path, image)
            dst = os.path.join(train_dir, class_name, image)
            shutil.move(src, dst)

        # Move the testing images to the test directory
        for image in test_images:
            src = os.path.join(subfolder_path, image)
            dst = os.path.join(test_dir, class_name, image)
            shutil.move(src, dst)

        print(f"Processed subfolder '{subfolder}' of class '{class_name}': {len(train_images)} train images, {len(test_images)} test images")

print("Dataset split complete!")


Processed subfolder 'Glass waste' of class 'glass': 0 train images, 0 test images
Processed subfolder 'Plastic' of class 'plastic': 0 train images, 0 test images
Processed subfolder 'paper waste' of class 'paper': 0 train images, 0 test images
Processed subfolder 'metal waste' of class 'metal': 0 train images, 0 test images
Dataset split complete!


In [42]:
PATH_TRAIN = r"/content/waste-classification-/content/waste-classification-/train_data"
PATH_TEST = r"/content/waste-classification-/content/waste-classification-/test_data"
class_names = ['glass', 'metal','paper','plastic']

# Check if the directories exist
if os.path.exists(PATH_TRAIN):
    print(f"Directory found: {PATH_TRAIN}")
else:
    print(f"Directory not found: {PATH_TRAIN}")

if os.path.exists(PATH_TEST):
    print(f"Directory found: {PATH_TEST}")
else:
    print(f"Directory not found: {PATH_TEST}")

Directory found: /content/waste-classification-/content/waste-classification-/train_data
Directory found: /content/waste-classification-/content/waste-classification-/test_data


In [44]:
import os
from PIL import Image

# Define the categories
class_names = ['glass', 'metal', 'paper', 'plastic']

# Define image paths for the 'paper' category
imagepath_paper = os.path.join(PATH_TRAIN, 'paper')
graypath_paper = os.path.join(PATH_TEST, 'paper')

# Ensure the output directory exists
os.makedirs(graypath_paper, exist_ok=True)

# Check if the directory exists
if os.path.exists(imagepath_paper):
    print(f"Directory found: {imagepath_paper}")
    file_listing = os.listdir(imagepath_paper)
else:
    print(f"Directory not found: {imagepath_paper}")

# Process each file in the 'paper' directory
for file in file_listing:
    # Ensure only image files are processed (e.g., jpg, png, gif)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Build the full path to the image file
        file_path = os.path.join(imagepath_paper, file)

        # Open the image
        im = Image.open(file_path)

        # If the image is a GIF with transparency, convert it to RGBA
        if im.mode == 'P' and 'transparency' in im.info:
            im = im.convert('RGBA')

        # Resize the image to 32x32
        img = im.resize((32, 32))

        # Convert the image to grayscale
        gray = img.convert('L')

        # Define the grayscale image file path
        gray_file_path = os.path.join(graypath_paper, file)

        # Save the grayscale image as JPEG
        gray.save(gray_file_path, "JPEG")
    else:
        print(f"Skipping non-image file: {file}")

print("Image processing completed successfully.")


Directory found: /content/waste-classification-/content/waste-classification-/train_data/paper
Image processing completed successfully.


In [45]:
import os
from PIL import Image

# Define the categories
class_names = ['glass', 'metal', 'paper', 'plastic']

# Define image paths for the 'glass' category
imagepath_glass = os.path.join(PATH_TRAIN, 'glass')
graypath_glass = os.path.join(PATH_TEST, 'glass')

# Ensure the output directory exists
os.makedirs(graypath_glass, exist_ok=True)

# Check if the directory exists
if os.path.exists(imagepath_glass):
    print(f"Directory found: {imagepath_glass}")
    file_listing = os.listdir(imagepath_glass)
else:
    print(f"Directory not found: {imagepath_glass}")

# Process each file in the 'glass' directory
for file in file_listing:
    # Ensure only image files are processed (e.g., jpg, png, gif)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Build the full path to the image file
        file_path = os.path.join(imagepath_glass, file)

        # Open the image
        im = Image.open(file_path)

        # If the image is a GIF with transparency, convert it to RGBA
        if im.mode == 'P' and 'transparency' in im.info:
            im = im.convert('RGBA')

        # Resize the image to 32x32
        img = im.resize((32, 32))

        # Convert the image to grayscale
        gray = img.convert('L')

        # Define the grayscale image file path
        gray_file_path = os.path.join(graypath_glass, file)

        # Save the grayscale image as JPEG
        gray.save(gray_file_path, "JPEG")
    else:
        print(f"Skipping non-image file: {file}")

print("Image processing completed successfully.")


Directory found: /content/waste-classification-/content/waste-classification-/train_data/glass
Skipping non-image file: Image_142.webp
Image processing completed successfully.


In [46]:
import os
from PIL import Image

# Define the categories
class_names = ['glass', 'metal', 'paper', 'plastic']

# Define image paths for the 'metal' category
imagepath_metal = os.path.join(PATH_TRAIN, 'metal')
graypath_metal = os.path.join(PATH_TEST, 'metal')

# Ensure the output directory exists
os.makedirs(graypath_metal, exist_ok=True)

# Check if the directory exists
if os.path.exists(imagepath_metal):
    print(f"Directory found: {imagepath_metal}")
    file_listing = os.listdir(imagepath_metal)
else:
    print(f"Directory not found: {imagepath_metal}")

# Process each file in the 'metal' directory
for file in file_listing:
    # Ensure only image files are processed (e.g., jpg, png, gif)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Build the full path to the image file
        file_path = os.path.join(imagepath_metal, file)

        # Open the image
        im = Image.open(file_path)

        # If the image is a GIF with transparency, convert it to RGBA
        if im.mode == 'P' and 'transparency' in im.info:
            im = im.convert('RGBA')

        # Resize the image to 32x32
        img = im.resize((32, 32))

        # Convert the image to grayscale
        gray = img.convert('L')

        # Define the grayscale image file path
        gray_file_path = os.path.join(graypath_metal, file)

        # Save the grayscale image as JPEG
        gray.save(gray_file_path, "JPEG")
    else:
        print(f"Skipping non-image file: {file}")

print("Image processing completed successfully.")


Directory found: /content/waste-classification-/content/waste-classification-/train_data/metal
Image processing completed successfully.


In [47]:
import os
from PIL import Image

# Define the categories
class_names = ['glass', 'metal', 'paper', 'plastic']

# Define image paths for the 'plastic' category
imagepath_plastic = os.path.join(PATH_TRAIN, 'plastic')
graypath_plastic = os.path.join(PATH_TEST, 'plastic')

# Ensure the output directory exists
os.makedirs(graypath_plastic, exist_ok=True)

# Check if the directory exists
if os.path.exists(imagepath_plastic):
    print(f"Directory found: {imagepath_plastic}")
    file_listing = os.listdir(imagepath_plastic)
else:
    print(f"Directory not found: {imagepath_plastic}")

# Process each file in the 'plastic' directory
for file in file_listing:
    # Ensure only image files are processed (e.g., jpg, png, gif)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Build the full path to the image file
        file_path = os.path.join(imagepath_plastic, file)

        # Open the image
        im = Image.open(file_path)

        # If the image is a GIF with transparency, convert it to RGBA
        if im.mode == 'P' and 'transparency' in im.info:
            im = im.convert('RGBA')

        # Resize the image to 32x32
        img = im.resize((32, 32))

        # Convert the image to grayscale
        gray = img.convert('L')

        # Define the grayscale image file path
        gray_file_path = os.path.join(graypath_plastic, file)

        # Save the grayscale image as JPEG
        gray.save(gray_file_path, "JPEG")
    else:
        print(f"Skipping non-image file: {file}")

print("Image processing completed successfully.")


Directory found: /content/waste-classification-/content/waste-classification-/train_data/plastic
Image processing completed successfully.


In [48]:
train_dir = os.path.join(PATH_TRAIN)
test_dir = os.path.join(PATH_TEST)

In [49]:
imagepath_paper_dir = os.path.join(imagepath_paper)
imagepath_glass_dir = os.path.join(imagepath_glass)
imagepath_metal_dir = os.path.join(imagepath_metal)
imagepath_plastic_dir = os.path.join(imagepath_plastic)

In [50]:
len(os.listdir(PATH_TRAIN))

4

In [51]:
IMG_HEIGHT = 32
IMG_WIDTH = 32
bath_size = 32 #to be tuned

In [52]:
#normalize to zero to one
image_gen = ImageDataGenerator(rescale=1./255)

In [53]:
train_data_gen = image_gen.flow_from_directory(
    directory = train_dir,
    shuffle=True,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

Found 443 images belonging to 4 classes.


In [54]:
test_data_gen = image_gen.flow_from_directory(
    directory = test_dir,
    shuffle=True,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

Found 634 images belonging to 4 classes.


In [55]:
len(os.listdir(PATH_TRAIN))

4